In [1]:
import json
import numpy as np
import os
from scipy.signal import find_peaks

from lif import lif_compute, spike_binary, id_synaptic_waveform
from ou_process import ouprocess_gaussian
from spike_sync import find_synced_spikes

In [11]:
NUM_NEURONS = 200
tau_V = 10
R = 1 # MOhm
EL = -70.0
V_th = -40.0
dt = 0.1 # msec
t_stop = 30.0e3

std_noise = 25

W = np.zeros((NUM_NEURONS, NUM_NEURONS))
train_input = None
train_exp_output = None

v_E = 0.0
v_ave = -67.0

tau_rise = 0.5
tau_fall = 5.0
syn_kernel_len = 50.0

_ETA = None

In [6]:
input_slow, _ = ouprocess_gaussian(50.0, dt, t_stop, 1)
i_inj = 16.0 + 6.0*input_slow

In [8]:
tt = np.arange(0.0, t_stop, dt)
V = np.zeros((tt.shape[0], NUM_NEURONS)) # Membrane potential per neuron

# Additive noise to individual neurons
if _ETA is None \
        or int_noise_regen is True\
        or _ETA.shape != V.shape:
    _ETA, _ = ouprocess_gaussian(5.0, dt, t_stop, NUM_NEURONS)

F_binary = np.zeros((tt.shape[0], NUM_NEURONS))
# avg_firing_rate = np.zeros(self.NUM_NEURONS)

I_total = std_noise*_ETA + i_inj

V = lif_compute(I_total, R, tau_V, V_th, dt)
F_binary = spike_binary(V)

syn_waveform = id_synaptic_waveform(
    dt,
    syn_kernel_len,
    tau_rise,
    tau_fall)
syn_wave_len = syn_waveform.shape[0]
t_steps = F_binary.shape[0]

F_synaptic = np.zeros(F_binary.shape)
for neuron in range(0, NUM_NEURONS):
    fr_fast = np.convolve(F_binary[:,neuron], syn_waveform)
    F_synaptic[:, neuron] = fr_fast[:-syn_wave_len+1]

ind_neur = np.arange(0, NUM_NEURONS)
Phi = F_synaptic[:t_steps, ind_neur]
X2 = -1.0*v_ave*np.ones((t_steps,ind_neur.shape[0])) + v_E

In [12]:
A = np.multiply(Phi, X2)
out = np.dot(A, W)

In [19]:
out.shape

(300000, 200)